In [1]:
import numpy as np
import pandas as pd
import torch
import os, sys
import time
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon, Ellipse
sys.path.append(os.path.realpath('./src/'))
from utilFuncs import to_np, to_torch
from materialEncoder import MaterialEncoder
from smallestEllipse import *
matplotlib.rcParams['figure.dpi'] = 150
matplotlib.rcParams['figure.figsize'] = (20, 10)

import seaborn as sns
import scipy as sp
%matplotlib qt

c:\Users\saket\.conda\envs\workSparse\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Solidworks database

In [2]:
def preprocessData():
  df = pd.read_excel('./data/bearingData.xlsx')# solidworksMaterialDatabaseCost # aluminum
  dataIdentifier = {'name': df[df.columns[0]], 'className':df[df.columns[1]], 'classID':df[df.columns[2]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[3:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  trainingData = normalizedData.clone().float()
  dataInfo = {'boreD':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'OD':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'Width':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'LoadRating':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]},\
              'RPM':{'idx':4,'scaleMin':dataScaleMin[4], 'scaleMax':dataScaleMax[4]},\
              'Weight':{'idx':5,'scaleMin':dataScaleMin[5], 'scaleMax':dataScaleMax[5]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

In [5]:
latentDim = 2 
hiddenDim = [500]
numEpochs = 20000
klFactor = 1e-5
learningRate = 2e-3
savedNet = './data/vaeNet.nt'
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}
materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)
# materialEncoder.loadAutoencoderFromFile(savedNet)
start = time.perf_counter()
convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
print('training time : {:.2F} '.format(time.perf_counter() - start))

cuda
Iter 0 reconLoss 5.80E+01 klLoss 7.53E-04 loss 5.80E+01
Learning Rate: 0.002
Iter 500 reconLoss 1.40E-01 klLoss 1.40E-02 loss 1.54E-01
Learning Rate: 0.002
Iter 1000 reconLoss 8.29E-02 klLoss 1.39E-02 loss 9.68E-02
Learning Rate: 0.002
Iter 1500 reconLoss 5.90E-02 klLoss 1.37E-02 loss 7.27E-02
Learning Rate: 0.002
Iter 2000 reconLoss 4.53E-02 klLoss 1.34E-02 loss 5.88E-02
Learning Rate: 0.002
Iter 2500 reconLoss 4.01E-02 klLoss 1.32E-02 loss 5.33E-02
Learning Rate: 0.002
Iter 3000 reconLoss 3.81E-02 klLoss 1.31E-02 loss 5.12E-02
Learning Rate: 0.002
Iter 3500 reconLoss 2.88E-02 klLoss 1.29E-02 loss 4.16E-02
Learning Rate: 0.002
Iter 4000 reconLoss 2.47E-02 klLoss 1.27E-02 loss 3.74E-02
Learning Rate: 0.002
Iter 4500 reconLoss 2.84E-02 klLoss 1.26E-02 loss 4.10E-02
Learning Rate: 0.002
Iter 5000 reconLoss 2.13E-02 klLoss 1.25E-02 loss 3.38E-02
Learning Rate: 0.002
Iter 5500 reconLoss 1.99E-02 klLoss 1.23E-02 loss 3.23E-02
Learning Rate: 0.002
Iter 6000 reconLoss 3.50E-02 klLoss 1.2

In [6]:
plt.close('all')


In [ ]:
from utilFuncs import plotConvergence
plotConvergence(convgHistory)

In [ ]:
print(trainInfo.shape)

(101, 6)


In [ ]:
def plotConvergence(convg):
  plt.figure();
  strokes = ['--', '-.', '-', ':']
  for ctr, key in enumerate(convg):
    y = torch.as_tensor(convg[key]).detach().numpy()
    y_mvavg = np.convolve(y, np.ones(20), 'valid') / 20.
    plt.semilogy(y_mvavg, strokes[ctr], label = str(key))
    plt.xlabel('Iterations')
    plt.ylabel(str(key))
    plt.grid('True')
    plt.legend()
    plt.savefig('./figures/convergence.pdf')

plotConvergence(convgHistory)

In [ ]:
matidxs = np.arange(trainInfo.shape[0]).astype(int)#0,11,20,46,54,69,91,5,92]
# matidxs = [13,14,15,48,18,10,9,8,24,25,20,30,69,27,37,4,5,6,73,77,78,85,91,88]
props = ['boreD','OD','Width','LoadRating','RPM','Weight']
# print([dataIdentifier['name'][i] for i in matidxs])
# print('\t \t ------TRUE DATA----------')
# print('Catalog Name', end = '\t')
# for p in props:
#     print(p, end = '\t')
# for i in matidxs:
#   print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
#   for p in props:
#     idx = materialEncoder.dataInfo[p]['idx']
#     print('\t {:.2E}'.format(10.**trainInfo[i,idx]),end='')

def unnormalize(val, minval ,maxval):
  return 10.**(minval + (maxval-minval)*val)
def decodeAll():
  vae = materialEncoder.vaeNet
  decoded = vae.decoder(vae.encoder.z)
  matProp = {'boreD':None,'OD':None,'Width':None,'LoadRating':None,'RPM':None,'Weight':None}

  for k in props:
    idx = materialEncoder.dataInfo[k]['idx']
    scaleMax = materialEncoder.dataInfo[k]['scaleMax']
    scaleMin = materialEncoder.dataInfo[k]['scaleMin']
    matProp[k] = unnormalize(decoded[:,idx], scaleMin ,scaleMax)#scaleMin + decoded[:,idx]*(scaleMax - scaleMin)
  return matProp

matProp = decodeAll()
# print('\n \n \t \t ------RECONSTRUCTED DATA----------') 
# print('Catalog Name', end = '\t')
# for p in props:
#     print(p, end = '\t')
  
# for i in matidxs:
#   print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
#   for p in props:
#     print('\t {:.2E}'.format(matProp[p][i]), end='')

merr = -1000000000.

maxError = {'boreD':merr,'OD':merr,'Width':merr,'LoadRating':merr,'RPM':merr,'Weight':merr}
print('\n \n \t \t ------RECON ERROR (%)----------') 
print('Catalog name', end = '\t')
errList = torch.zeros(trainInfo.shape[0],6);
for p in props:
    print(p, end = '\t')
for i in range(trainInfo.shape[0]):
  count = 0;

  if(i in matidxs): #
    print(f"\n {dataIdentifier['name'][i]} \t ", end = '')

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    trueData = 10**trainInfo[i,idx]
    reconData = matProp[p][i]
    err = torch.abs(100.*(trueData - reconData)/trueData).to('cpu')
    errList[i,count] = err
    count = count + 1;
    if(err > maxError[p]):
      maxError[p] = err
    if(i in matidxs):
      print('\t {:.1F}'.format(err), end='')
  
      
print('\n max Error', end = '')
for p in props:
  print('\t {:.1F}'.format(maxError[p]), end='')

print("\n Mean Error:")
print(torch.mean(errList,0))


 
 	 	 ------RECON ERROR (%)----------
Catalog name	boreD	OD	Width	LoadRating	RPM	Weight	
 1 	 	 0.4	 0.8	 0.0	 0.7	 0.6	 0.0
 44 	 	 0.6	 0.4	 0.2	 1.8	 0.5	 0.2
 69 	 	 0.5	 0.5	 0.3	 0.6	 0.6	 0.0
 85 	 	 0.2	 0.3	 0.2	 0.8	 0.3	 3.4
 2 	 	 0.2	 0.6	 0.3	 2.6	 0.6	 0.6
 3 	 	 1.1	 0.1	 0.1	 1.1	 0.3	 0.7
 4 	 	 0.6	 0.4	 0.3	 1.2	 0.9	 0.8
 5 	 	 0.8	 0.1	 0.1	 3.1	 0.0	 0.3
 6 	 	 1.1	 0.2	 0.4	 0.7	 0.4	 1.6
 7 	 	 0.2	 0.2	 0.1	 0.5	 0.7	 0.4
 8 	 	 1.4	 0.0	 0.5	 1.5	 1.7	 0.2
 9 	 	 0.3	 0.5	 0.1	 2.3	 2.2	 0.6
 10 	 	 1.9	 0.1	 1.0	 0.1	 0.4	 0.7
 11 	 	 0.6	 0.2	 0.1	 2.6	 0.0	 0.0
 12 	 	 0.7	 0.0	 0.1	 3.9	 0.4	 1.2
 13 	 	 0.1	 0.3	 0.1	 2.3	 0.6	 2.0
 14 	 	 0.4	 0.1	 0.0	 0.7	 0.3	 0.0
 15 	 	 1.1	 0.1	 0.5	 2.0	 2.3	 2.6
 16 	 	 0.3	 0.4	 0.0	 0.1	 1.6	 1.5
 17 	 	 1.9	 0.6	 0.3	 1.6	 3.3	 0.8
 18 	 	 0.5	 1.3	 0.7	 1.6	 1.8	 1.0
 19 	 	 0.0	 0.2	 0.2	 0.0	 0.3	 1.2
 20 	 	 0.6	 0.1	 0.3	 1.1	 0.7	 1.2
 21 	 	 1.4	 0.7	 0.4	 1.4	 0.4	 0.0
 22 	 	 0.9	 0.8	 0.1	 5.0	 0.

In [7]:
def plotLatent(ltnt1, ltnt2, plotHull, plotEllipse, annotateHead, saveFileName):
    clrs = ['purple', 'green', 'red', 'blue', 'black', 'violet', 'cyan']
    mrkrSet = ['x','D','s','p','*','o','P']
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    fig, ax = plt.subplots()
    # matidxs = np.array([13,14,15,48,18,10,9,8,24,20,30,69,27,37,5,6,73,77,78,85,91,88,75,80,82]).astype(int)-2
    # matidxs = np.arange(trainInfo.shape[0]).astype(int)
    for i in range(np.max(colorcol)+1): 
      zMat = np.vstack((z[colorcol == i,ltnt1], z[colorcol == i,ltnt2])).T
      ax.scatter(zMat[:, 0], zMat[:, 1], marker=mrkrSet[i], c = clrs[i], s = 12)#clrs[i]

      if(plotHull):
        hull = ConvexHull(zMat)
        cent = np.mean(zMat, 0)
        pts = []
        for pt in zMat[hull.simplices]:
            pts.append(pt[0].tolist())
            pts.append(pt[1].tolist())
  
        pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                        p[0] - cent[0]))
        pts = pts[0::2]  # Deleting duplicates
        pts.insert(len(pts), pts[0])
        # print(pts)
        poly = Polygon(1.1*(np.array(pts)- cent) + cent,
                       facecolor= clrs[i], alpha=0.2, edgecolor = 'black') #'black'
        poly.set_capstyle('round')
        plt.gca().add_patch(poly)
        # ax.annotate(dataIdentifier['className'][i], (cent[0], cent[1]), size = 15, c = 'red')
        # print(dataIdentifier['className'][i])

      if(plotEllipse):
        hull = ConvexHull(zMat)
        cent = np.mean(zMat, 0)
        pts = []
        for pt in zMat[hull.simplices]:
            pts.append(pt[0].tolist())
            pts.append(pt[1].tolist())
  
        pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                        p[0] - cent[0]))
        pts = pts[0::2]  # Deleting duplicates
        # pts.insert(len(pts), pts[0])
        enclosing_ellipse = welzl(np.array(pts, dtype=float))
        # plot resulting ellipse
        center,a,b,t = enclosing_ellipse
        elli = plot_ellipse(enclosing_ellipse, str='k')
        ellipse = Ellipse(xy=center, width=2*a, height=2*b, angle=np.degrees(t), edgecolor='k', fc=clrs[i], alpha=0.3, lw=2)
        ax.add_patch(ellipse)

        ax.annotate('Deep Groove Ball Bearings', xy=(-1.5, 0.5), xytext=(-3,1), size = 14, c = 'black', xycoords='data',textcoords='data',arrowprops=dict(arrowstyle="-"))
        ax.annotate('Angular Contact Ball Bearings', xy=(-0.5, -0.5) ,xytext=(0.5,-1.5), size = 14, c = 'black' ,xycoords='data',textcoords='data',arrowprops=dict(arrowstyle="-"))
        ax.annotate('Cylindrical Roller Bearings', xy=(0.2,-0.2), xytext=(0.8,-0.5),size = 14, c = 'black', xycoords='data',textcoords='data',arrowprops=dict(arrowstyle="-"))
        ax.annotate('Thrust Bearings',xy=(1,1.5),xytext=(1,2), size = 14, c = 'black', xycoords='data',textcoords='data',arrowprops=dict(arrowstyle="-"))
        plt.show()

        # poly = Polygon(1.1*(np.array(pts)- cent) + cent,
        #                facecolor= clrs[i], alpha=0.2, edgecolor = 'black') #'black'
        # poly.set_capstyle('round')

        # plt.gca().add_patch(poly)
        # ax.annotate(dataIdentifier['className'][i], (cent[0], cent[1]), size = 15, c = 'red')
        # print(dataIdentifier['className'][i])
        # print(i)

    matidxs = [ ] 
    for i, txt in enumerate(ptLabel):
      if(annotateHead == False or ( annotateHead == True and  i in matidxs)):
        
        ax.annotate(txt, (z[i,ltnt1], z[i,ltnt2]), size = 10)
        ax.scatter(z[i,ltnt1], z[i,ltnt2], marker='*', c = 'red', s = 56)

  #   plt.axis('off')
    ticks = [-3, -2,  -1., 0.,  1., 2, 3]
    ticklabels = ['-3','-2', '-1', '0','1', '2','3']
    plt.xticks(ticks, ticklabels, fontsize=18)
    plt.yticks(ticks, ticklabels, fontsize=18)
    plt.xlabel('z{:d}'.format(ltnt1), size = 18)
    plt.ylabel('z{:d}'.format(ltnt2), size = 18)
    minor_ticks = np.arange(-3, 3, 0.1)
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(minor_ticks, minor=True)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_aspect('equal', 'box')

    # plt.grid(which='minor')
    plt.grid(visible=None)
    plt.savefig(saveFileName,bbox_inches='tight')
    
    return fig, ax
  
# plotLatent(0, 1, plotHull = True, plotEllipse = False, annotateHead = True, saveFileName = './figures/latent.pdf')

In [10]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)
plotLatent(0, 1, plotHull = False, plotEllipse = True, annotateHead = True, saveFileName = './figures/Bearinglatent.pdf')

(<Figure size 1924x1027 with 1 Axes>, <AxesSubplot:xlabel='z0', ylabel='z1'>)

In [11]:
def plotLatentWithPropertyNew(ltnt1 = 0, ltnt2 = 1):
  n = 80
  zmin, zmax = -3,3
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim'])).to('cpu')
  Z[:,ltnt1], Z[:,ltnt2] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet.to('cpu')
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)



  #-------------------------------------------#
  props = ['RPM']
  cutOff = [2,5]; 

  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = to_np(10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin)))
    levs = np.logspace(np.log10(min(matPropVal))*0.5, np.log10(max(matPropVal)), 40)
    fig, ax = plotLatent(0, 1, plotHull = False, plotEllipse = True, annotateHead = True, saveFileName = './figures/Bearinglatent.pdf')
    surf = ax.contour(X, Y, (matPropVal.reshape((n,n))), levels = levs, cmap='viridis_r', alpha = 0.6)

    # surf = ax.contour(X, Y, (to_np(matPropVal).reshape((n,n))), levels = cutOff, cmap='coolwarm', alpha = 0.3)

    # surf = ax.contourf(X, Y, (to_np(matPropVal).reshape((n,n))), levels = cutOff, alpha = 0.2,\
    # colors=['g', 'g', '#C0C0C0'], extend='both')
    # surf.cmap.set_over('white')
    # surf.cmap.set_under('white')
    # surf.changed()

    

    plt.clabel(surf, inline=False, fontsize=12, fmt ='%0.2f', colors = 'black')
    ax.set_xlabel('$z_0$')
    ax.set_ylabel('$z_1$')
    ax.set_title(p)
    cbar = plt.colorbar(surf)
    cbar.set_label('({:s})'.format(str(p)))
    plt.show()
    plt.savefig('./figures/{:s}_latentFieldContours.pdf'.format(p), dpi=200, bbox_inches='tight')

  #-------------------------------------------#
  

plt.close('all')
plotLatentWithPropertyNew()

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


In [27]:
plt.close('all')

In [ ]:
def plotTrueAndReconstructedDistribution():

  vae = materialEncoder.vaeNet
  trainData_z_np = to_np(vae.encoder.z)
  decodedVals = vae.decoder(vae.encoder.z)
  
  bw = 0.405
  fig, ax = plt.subplots(1,2)
  #-------------------------------------------#
  props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
  props = ['youngsModulus','yieldStrength']
  for ctr, p in enumerate(props):
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matVal_decoded = 10.**(scaleMin + decodedVals[:,idx]*(scaleMax - scaleMin))
    matVal_data = 10.**(scaleMin + trainingData[:,idx]*(scaleMax - scaleMin))

    sns.set_style('whitegrid')
    plt.subplot(1,2,ctr+1)
    f = sns.kdeplot(to_np(matVal_decoded), bw_adjust=bw, fill = True, alpha = 0.1, label='decoded')
    f = sns.kdeplot(to_np(matVal_data), bw_adjust=bw,  fill = True, alpha = 0.1, linestyle="--", label='actual')
    f.set(xlabel = p, ylabel = 'frequency',yticklabels=[])
    plt.legend()
    plt.axis('auto')
    plt.title(p)
  
  plt.savefig('./figures/trueAndReconstructedDistribution.pdf'.format(p), dpi=200, bbox_inches='tight')

plotTrueAndReconstructedDistribution()  

In [ ]:
def plotLatentPropertyWithGradients(ltnt1 = 0, ltnt2 = 1):
  n = 40
  zmin, zmax = -2.5,2.5
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim']))
  Z[:,ltnt1], Z[:,ltnt2] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))
  Z = torch.tensor(Z, requires_grad = True)
  vae = materialEncoder.vaeNet
  decodedVals = vae.decoder(Z)



  fig, ax = plt.subplots(1,1)
  #-------------------------------------------#
  props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
  props = ['yieldStrength']
  for ctr, p in enumerate(props):
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matVal_decoded = 10.**(scaleMin + decodedVals[:,idx]*(scaleMax - scaleMin))

    dE_dz = to_np(torch.autograd.grad(matVal_decoded, Z, grad_outputs = torch.ones(Z.shape[0]), create_graph = True)[0])
    U = dE_dz[:,0] / (1e-4+np.sqrt(dE_dz[:,0]**2 + dE_dz[:,1]**2))
    V = dE_dz[:,1] / (1e-4+np.sqrt(dE_dz[:,0]**2 + dE_dz[:,1]**2))
    plt.subplot(1,1,ctr+1)
    surf = plt.contourf(X, Y, np.log10(to_np(matVal_decoded).reshape((n,n))), levels = 100, cmap='coolwarm', alpha = 0.7)
    plt.quiver(X,Y,U,V, headwidth = 0, headlength = 0,headaxislength = 0, color = 'black')
#     plt.clabel(surf, inline=False, fontsize=12, fmt ='%0.2F', colors = 'black')
    plt.title(p)
    plt.xlabel('$z_0$')
    plt.ylabel('$z_1$')
    cbar = plt.colorbar(surf)
#     cbar.set_label('$log_{10}$({:s})'.format(p))
  
  plt.savefig('./figures/latentSpaceGradient.pdf'.format(p), dpi=200, bbox_inches='tight')
  
plotLatentPropertyWithGradients()